# <ins>Project 1B - Data Modeling with Cassandra</ins>

## Part I. ETL Pipeline for Pre-Processing the Files

PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

### Import Packages

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import sys
import platform
print(f"Executable Python: {sys.executable}")
print(f"Python Version: {platform.python_version()}")

Executable Python: /home/siddharth/virtual_envs/smaity_env1/bin/python3
Python Version: 3.8.5


### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/home/siddharth/Documents/GitHub/Udacity_Data_Engineering_ND/Data_Modeling/3_NoSQL_Data_Models/Project_2_Data_Modeling_with_Cassandra


### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

## Part II. Complete the Apache Cassandra coding portion of your project. 

### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Creating a Cluster

In [4]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS smaity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### Set Keyspace

In [6]:
try:
    session.set_keyspace('smaity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Create tables to run the above queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Query 1

In [7]:
## Setup Query
query = "CREATE TABLE IF NOT EXISTS session_item_table"
query = query + "(artist text, song text, length float, sessionid int, iteminsession int, PRIMARY KEY (sessionid, iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
# Insert Data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item_table (artist, song, length, sessionid, iteminsession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [9]:
# Do a SELECT to verify that the data have been inserted into each table
# Query - Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query_test = "select artist, song, length from session_item_table where sessionid = 338 and iteminsession = 4"
try:
    rows = session.execute(query_test)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [10]:
# Pulling csv into dataframe to confirm all rows were retrieved by the query above
df_test = pd.read_csv("event_datafile_new.csv")
df_test[(df_test["sessionId"]==338) & (df_test["itemInSession"]==4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4219,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


## Query 2

In [11]:
## Setup Query
query = "CREATE TABLE IF NOT EXISTS user_session_table"
query = query + "(artist text, song text, first_name text, last_name text, user_id int, session_id int, iteminsession int, PRIMARY KEY (user_id, session_id, iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Insert Data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_table (artist, song, first_name, last_name, user_id, session_id, iteminsession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [13]:
# Do a SELECT to verify that the data have been inserted into each table
# Query - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query_test = "select artist, song, first_name, last_name from user_session_table where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query_test)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
# Pulling csv into dataframe to confirm all rows were retrieved by the query above
df_test = pd.read_csv("event_datafile_new.csv")
df_test[(df_test["userId"]==10) & (df_test["sessionId"]==182)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4705,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4706,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4707,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4708,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


## Query 3

In [15]:
## Setup Query
query = "CREATE TABLE IF NOT EXISTS song_table"
query = query + "(first_name text, last_name text, song text, PRIMARY KEY (song, first_name, last_name))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# Insert Data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_table (first_name, last_name, song)"
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9]))

In [17]:
# Do a SELECT to verify that the data have been inserted into each table
# Query - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query_test = "select first_name, last_name from song_table where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query_test)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)


Jacqueline Lynch
Sara Johnson
Tegan Levine


In [18]:
# Pulling csv into dataframe to confirm all rows were retrieved by the query above
df_test = pd.read_csv("event_datafile_new.csv")
df_test[(df_test["song"]=='All Hands Against His Own')]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2326,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
4984,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
6418,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95


## Drop Tables

In [19]:
query = "drop table if exists session_item_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table if exists user_session_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table if exists song_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Close Session and cluster connection

In [20]:
session.shutdown()
cluster.shutdown()